In [7]:
import tensorflow as tf
from tensorflow.keras import Model, layers, losses, optimizers
from tensorflow.keras.applications import VGG16
import numpy as np
import matplotlib.pyplot as plt

def conv_block(inputs, filters, dropout_rate=0.3):
    x = layers.Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def unet_model(input_shape=(256, 256, 3), num_classes=21):
    inputs = layers.Input(shape=input_shape)
    
    # Encoder (Contracting Path)
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    
    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)
    
    # Bridge
    bridge = conv_block(p4, 1024)
    
    # Decoder (Expansive Path)
    u1 = layers.UpSampling2D((2, 2))(bridge)
    u1 = layers.concatenate([u1, c4])
    c5 = conv_block(u1, 512)
    
    u2 = layers.UpSampling2D((2, 2))(c5)
    u2 = layers.concatenate([u2, c3])
    c6 = conv_block(u2, 256)
    
    u3 = layers.UpSampling2D((2, 2))(c6)
    u3 = layers.concatenate([u3, c2])
    c7 = conv_block(u3, 128)
    
    u4 = layers.UpSampling2D((2, 2))(c7)
    u4 = layers.concatenate([u4, c1])
    c8 = conv_block(u4, 64)
    
    # Output
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(c8)
    
    return Model(inputs, outputs)

# Compilation du modèle
model = unet_model()
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [8]:
class AttentionGate(layers.Layer):
    def __init__(self, filters):
        super(AttentionGate, self).__init__()
        self.filters = filters
        
    def build(self, input_shape):
        self.w_g = layers.Conv2D(self.filters, 1, padding='same')
        self.w_x = layers.Conv2D(self.filters, 1, padding='same')
        self.psi = layers.Conv2D(1, 1, padding='same', activation='sigmoid')
        
    def call(self, inputs):
        g, x = inputs
        g1 = self.w_g(g)
        x1 = self.w_x(x)
        psi = tf.nn.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

def attention_unet(input_shape=(256, 256, 3), num_classes=21):
    inputs = layers.Input(shape=input_shape)
    
    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    
    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)
    
    # Bridge
    bridge = conv_block(p4, 1024)
    
    # Decoder with Attention
    u1 = layers.UpSampling2D((2, 2))(bridge)
    att1 = AttentionGate(512)([u1, c4])
    u1 = layers.concatenate([u1, att1])
    c5 = conv_block(u1, 512)
    
    u2 = layers.UpSampling2D((2, 2))(c5)
    att2 = AttentionGate(256)([u2, c3])
    u2 = layers.concatenate([u2, att2])
    c6 = conv_block(u2, 256)
    
    u3 = layers.UpSampling2D((2, 2))(c6)
    att3 = AttentionGate(128)([u3, c2])
    u3 = layers.concatenate([u3, att3])
    c7 = conv_block(u3, 128)
    
    u4 = layers.UpSampling2D((2, 2))(c7)
    att4 = AttentionGate(64)([u4, c1])
    u4 = layers.concatenate([u4, att4])
    c8 = conv_block(u4, 64)
    
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(c8)
    
    return Model(inputs, outputs)

In [9]:
def resnet_unet(input_shape=(256, 256, 3), num_classes=21):
    base_model = tf.keras.applications.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Utiliser les couches spécifiques pour les skip connections
    layer_names = [
        'conv1_relu',          # 64 filters
        'conv2_block3_out',    # 256 filters
        'conv3_block4_out',    # 512 filters
        'conv4_block6_out',    # 1024 filters
    ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
    
    # Create feature extraction model
    encoder = Model(inputs=base_model.input, outputs=base_model_outputs)
    encoder.trainable = False  # Transfer learning
    
    # Decoder
    inputs = layers.Input(shape=input_shape)
    features = encoder(inputs)
    
    # Reverse features for decoder
    f1, f2, f3, f4 = features
    
    # Decoder with skip connections
    x = layers.Conv2DTranspose(512, 3, strides=2, padding='same')(f4)
    x = layers.concatenate([x, f3])
    x = conv_block(x, 512)
    
    x = layers.Conv2DTranspose(256, 3, strides=2, padding='same')(x)
    x = layers.concatenate([x, f2])
    x = conv_block(x, 256)
    
    x = layers.Conv2DTranspose(128, 3, strides=2, padding='same')(x)
    x = layers.concatenate([x, f1])
    x = conv_block(x, 128)
    
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = conv_block(x, 64)
    
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(x)
    
    return Model(inputs, outputs)

In [10]:
def deeplabv3_plus(input_shape=(512, 512, 3), num_classes=21, output_stride=16):
    # Backbone Xception
    base_model = tf.keras.applications.Xception(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Atrous Spatial Pyramid Pooling
    def aspp_block(inputs, filters=256):
        # Different dilation rates
        conv1 = layers.Conv2D(filters, 1, padding='same')(inputs)
        conv2 = layers.Conv2D(filters, 3, padding='same', dilation_rate=6)(inputs)
        conv3 = layers.Conv2D(filters, 3, padding='same', dilation_rate=12)(inputs)
        conv4 = layers.Conv2D(filters, 3, padding='same', dilation_rate=18)(inputs)
        
        # Global Average Pooling
        gap = layers.GlobalAveragePooling2D()(inputs)
        gap = layers.Reshape((1, 1, filters))(gap)
        gap = layers.Conv2D(filters, 1, padding='same')(gap)
        gap = layers.UpSampling2D(size=(input_shape[0]//output_stride, 
                                       input_shape[1]//output_stride))(gap)
        
        # Concatenate all branches
        return layers.concatenate([conv1, conv2, conv3, conv4, gap])
    
    # Encoder
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    
    # ASPP
    aspp = aspp_block(x)
    aspp = layers.Conv2D(256, 1, padding='same')(aspp)
    aspp = layers.BatchNormalization()(aspp)
    aspp = layers.Activation('relu')(aspp)
    
    # Decoder
    decoder = layers.UpSampling2D((4, 4))(aspp)
    decoder = layers.concatenate([decoder, base_model.get_layer('block13_sepconv2_bn').output])
    decoder = layers.Conv2D(256, 3, padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    
    decoder = layers.UpSampling2D((4, 4))(decoder)
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(decoder)
    
    return Model(inputs, outputs)

In [11]:
class GAN:
    def __init__(self, img_shape=(64, 64, 3), latent_dim=100):
        self.img_shape = img_shape
        self.latent_dim = latent_dim
        
        # Build models
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()
        
        # Compile discriminator
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=optimizers.Adam(0.0002, 0.5),
            metrics=['accuracy']
        )
        
        # Combined model
        z = layers.Input(shape=(self.latent_dim,))
        img = self.generator(z)
        self.discriminator.trainable = False
        validity = self.discriminator(img)
        
        self.combined = Model(z, validity)
        self.combined.compile(
            loss='binary_crossentropy',
            optimizer=optimizers.Adam(0.0002, 0.5)
        )
    
    def build_generator(self):
        model = tf.keras.Sequential([
            layers.Dense(8*8*256, input_dim=self.latent_dim),
            layers.Reshape((8, 8, 256)),
            layers.Conv2DTranspose(128, 4, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Conv2DTranspose(64, 4, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Conv2DTranspose(32, 4, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Conv2D(3, 3, padding='same', activation='tanh')
        ])
        return model
    
    def build_discriminator(self):
        model = tf.keras.Sequential([
            layers.Conv2D(32, 3, strides=2, input_shape=self.img_shape, padding='same'),
            layers.LeakyReLU(0.2),
            layers.Dropout(0.25),
            layers.Conv2D(64, 3, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dropout(0.25),
            layers.Conv2D(128, 3, strides=2, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dropout(0.25),
            layers.Conv2D(256, 3, strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2),
            layers.Dropout(0.25),
            layers.Flatten(),
            layers.Dense(1, activation='sigmoid')
        ])
        return model

In [14]:
class SelfAttention(layers.Layer):
    def __init__(self, filters):
        super(SelfAttention, self).__init__()
        self.filters = filters
        self.gamma = self.add_weight(shape=[1], initializer='zeros')
        
    def build(self, input_shape):
        self.query = layers.Conv2D(self.filters//8, 1)
        self.key = layers.Conv2D(self.filters//8, 1)
        self.value = layers.Conv2D(self.filters, 1)
        
    def call(self, x):
        batch_size, h, w, c = x.shape
        
        # Query, Key, Value
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        # Reshape for matrix multiplication
        q = tf.reshape(q, [batch_size, h*w, -1])
        k = tf.reshape(k, [batch_size, h*w, -1])
        v = tf.reshape(v, [batch_size, h*w, -1])
        
        
        # Attention scores
        attention = tf.matmul(q, k, transpose_b=True)
        attention = tf.nn.softmax(attention, axis=-1)
        
        # Weighted sum
        out = tf.matmul(attention, v)
        out = tf.reshape(out, [batch_size, h, w, -1])
        
        return self.gamma * out + x